# 🚗 Nhận Dạng Biển Số Xe Việt Nam
## Vietnamese License Plate Recognition using YOLOv5

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ngoclam99/BsxGoogleColab/blob/main/License_Plate_Recognition_Colab.ipynb)

### 📋 Tính năng:
- 🔍 Phát hiện biển số xe trong ảnh
- 📝 Nhận dạng ký tự và số trên biển số
- 🎯 Hỗ trợ biển số 1 dòng và 2 dòng
- 📊 Hiển thị kết quả trực quan

### 🚀 Hướng dẫn sử dụng:
1. **Chọn GPU Runtime**: Runtime → Change runtime type → Hardware accelerator → GPU
2. **Chạy từng cell theo thứ tự** từ trên xuống dưới
3. **Đợi cell hoàn thành** trước khi chạy cell tiếp theo
4. **Upload ảnh của bạn** hoặc sử dụng ảnh mẫu có sẵn

---

## 🔧 1. Cài đặt và Thiết lập

In [ ]:
import os
import sys

# Kiểm tra xem đã clone repository chưa
if not os.path.exists('BsxGoogleColab'):
    print("🔄 Đang clone repository từ GitHub...")
    !git clone https://github.com/ngoclam99/BsxGoogleColab.git
    print("✅ Clone thành công!")
else:
    print("✅ Repository đã tồn tại!")

# Chuyển đến thư mục project
os.chdir('BsxGoogleColab')
print(f"📁 Thư mục hiện tại: {os.getcwd()}")

# Thêm thư mục vào Python path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

In [ ]:
# Cài đặt dependencies
print("📦 Đang cài đặt dependencies...")
!pip install -q opencv-python-headless matplotlib numpy Pillow torch torchvision tqdm pyyaml requests pandas seaborn

# Cài đặt thêm các package cần thiết cho Colab
!pip install -q ipywidgets

print("✅ Cài đặt hoàn tất!")

## 📚 2. Import Libraries và Kiểm tra Files

In [ ]:
# Import các thư viện cần thiết
import cv2
import torch
import math
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Import helper functions
try:
    import function.utils_rotate as utils_rotate
    import function.helper as helper
    print("✅ Import helper functions thành công!")
except ImportError as e:
    print(f"❌ Lỗi import helper functions: {e}")
    print("💡 Kiểm tra lại cấu trúc thư mục")

# Thiết lập matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Import libraries thành công!")

In [ ]:
# Kiểm tra cấu trúc thư mục và files quan trọng
print("🔍 Kiểm tra cấu trúc thư mục...")

required_files = {
    'YOLOv5 Directory': 'yolov5',
    'LP Detector Model': 'model/LP_detector.pt',
    'LP OCR Model': 'model/LP_ocr.pt',
    'Helper Functions': 'function/helper.py',
    'Utils Rotate': 'function/utils_rotate.py',
    'Test Images': 'test_image'
}

all_files_exist = True
for name, path in required_files.items():
    if os.path.exists(path):
        if os.path.isfile(path):
            size = os.path.getsize(path) / (1024*1024)  # MB
            print(f"✅ {name}: {path} ({size:.1f} MB)")
        else:
            files_count = len(os.listdir(path)) if os.path.isdir(path) else 0
            print(f"✅ {name}: {path} ({files_count} files)")
    else:
        print(f"❌ {name}: {path} - KHÔNG TỒN TẠI")
        all_files_exist = False

if all_files_exist:
    print("\n🟢 Tất cả files cần thiết đã sẵn sàng!")
else:
    print("\n🔴 Một số files quan trọng bị thiếu!")
    print("💡 Hãy kiểm tra lại repository hoặc push lại files lên GitHub")

## 🤖 3. Load YOLOv5 Models

In [ ]:
# Load YOLOv5 models
print("🤖 Đang load YOLOv5 models...")
print("⏳ Quá trình này có thể mất 2-3 phút lần đầu...")

# Khởi tạo biến models
yolo_LP_detect = None
yolo_license_plate = None

try:
    # Load model phát hiện biển số
    print("📥 Đang load License Plate Detection Model...")
    yolo_LP_detect = torch.hub.load('yolov5', 'custom', 
                                   path='model/LP_detector.pt', 
                                   force_reload=True, 
                                   source='local')
    print("✅ License Plate Detection Model loaded!")
    
    # Load model nhận dạng ký tự
    print("📥 Đang load License Plate OCR Model...")
    yolo_license_plate = torch.hub.load('yolov5', 'custom', 
                                       path='model/LP_ocr.pt', 
                                       force_reload=True, 
                                       source='local')
    print("✅ License Plate OCR Model loaded!")
    
    # Thiết lập confidence threshold
    yolo_license_plate.conf = 0.60
    
    # Test models với ảnh dummy
    print("🧪 Testing models...")
    dummy_img = np.zeros((640, 640, 3), dtype=np.uint8)
    _ = yolo_LP_detect(dummy_img)
    _ = yolo_license_plate(dummy_img)
    
    print("\n🎉 TẤT CẢ MODELS ĐÃ SẴN SÀNG!")
    print("🟢 Có thể bắt đầu nhận dạng biển số")
    
except Exception as e:
    print(f"\n❌ LỖI KHI LOAD MODELS: {e}")
    print("\n💡 CÁCH KHẮC PHỤC:")
    print("   1. Restart Runtime: Runtime → Restart Runtime")
    print("   2. Chạy lại từ đầu")
    print("   3. Kiểm tra kết nối internet")
    print("   4. Đảm bảo đã chọn GPU Runtime")

## 🔧 4. Định nghĩa hàm nhận dạng

In [ ]:
def detect_license_plate(image_path, show_details=True, save_result=False):
    """
    Nhận dạng biển số xe từ ảnh
    
    Args:
        image_path (str): Đường dẫn đến file ảnh
        show_details (bool): Hiển thị chi tiết quá trình
        save_result (bool): Lưu ảnh kết quả
    
    Returns:
        tuple: (ảnh kết quả, danh sách biển số phát hiện)
    """
    
    # Kiểm tra models
    if yolo_LP_detect is None or yolo_license_plate is None:
        print("❌ Models chưa được load! Hãy chạy cell load models trước.")
        return None, None
    
    if show_details:
        print(f"🔍 Đang xử lý: {image_path}")
    
    # Đọc ảnh
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Không thể đọc ảnh: {image_path}")
        return None, None
    
    original_img = img.copy()
    
    try:
        if show_details:
            print("📸 Đang phát hiện biển số...")
        
        # Phát hiện biển số
        plates = yolo_LP_detect(img, size=640)
        list_plates = plates.pandas().xyxy[0].values.tolist()
        list_read_plates = set()
        
        if show_details:
            print(f"🎯 Phát hiện {len(list_plates)} vùng biển số")
        
        if len(list_plates) == 0:
            # Thử đọc toàn bộ ảnh nếu không phát hiện được biển số
            if show_details:
                print("🔄 Thử nhận dạng trên toàn bộ ảnh...")
            lp = helper.read_plate(yolo_license_plate, img)
            if lp != "unknown":
                list_read_plates.add(lp)
        else:
            # Xử lý từng biển số
            for i, plate in enumerate(list_plates):
                if show_details:
                    print(f"📝 Đang đọc biển số {i+1}/{len(list_plates)}...")
                
                x = int(plate[0])
                y = int(plate[1])
                w = int(plate[2] - plate[0])
                h = int(plate[3] - plate[1])
                
                # Cắt ảnh biển số
                crop_img = img[y:y+h, x:x+w]
                
                # Vẽ khung biển số
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
                
                # Thử các cách xử lý ảnh khác nhau
                found = False
                for cc in range(0, 2):
                    for ct in range(0, 2):
                        processed_img = utils_rotate.deskew(crop_img, cc, ct)
                        lp = helper.read_plate(yolo_license_plate, processed_img)
                        if lp != "unknown":
                            list_read_plates.add(lp)
                            found = True
                            break
                    if found:
                        break
        
        # Thêm text lên ảnh
        if list_read_plates:
            y_offset = 30
            for plate_text in list_read_plates:
                cv2.putText(img, plate_text, (10, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                y_offset += 40
        
        if show_details:
            if list_read_plates:
                print(f"✅ Kết quả: {list_read_plates}")
            else:
                print("❌ Không nhận dạng được biển số")
        
        # Lưu kết quả nếu cần
        if save_result and list_read_plates:
            result_path = f"result_{os.path.basename(image_path)}"
            cv2.imwrite(result_path, img)
            print(f"💾 Đã lưu kết quả: {result_path}")
        
        return img, list_read_plates
        
    except Exception as e:
        print(f"❌ Lỗi trong quá trình xử lý: {e}")
        return None, None

def display_results(original_path, result_img, detected_plates):
    """
    Hiển thị kết quả nhận dạng
    """
    if result_img is None:
        print("❌ Không có kết quả để hiển thị")
        return
    
    # Tạo subplot
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Ảnh gốc
    original_img = cv2.imread(original_path)
    axes[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title("🖼️ Ảnh gốc", fontsize=16, fontweight='bold')
    axes[0].axis('off')
    
    # Ảnh kết quả
    axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    result_text = f"🎯 Kết quả: {detected_plates if detected_plates else 'Không nhận dạng được'}"
    axes[1].set_title(result_text, fontsize=16, fontweight='bold')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Thông tin chi tiết
    print(f"\n📊 THÔNG TIN CHI TIẾT:")
    print(f"   📁 File: {original_path}")
    print(f"   📏 Kích thước: {original_img.shape[1]}x{original_img.shape[0]}")
    print(f"   🔢 Số biển số: {len(detected_plates) if detected_plates else 0}")
    if detected_plates:
        for i, plate in enumerate(detected_plates, 1):
            print(f"   🚗 Biển số {i}: {plate}")

print("✅ Các hàm nhận dạng đã sẵn sàng!")

## 🖼️ 5. Test với ảnh mẫu

In [ ]:
# Liệt kê ảnh test có sẵn
print("📁 Tìm kiếm ảnh test...")
test_images = glob.glob("test_image/*.jpg") + glob.glob("test_image/*.png") + glob.glob("test_image/*.jpeg")

if test_images:
    print(f"🖼️ Tìm thấy {len(test_images)} ảnh test:")
    for i, img_path in enumerate(test_images):
        print(f"   {i}: {img_path}")
else:
    print("❌ Không tìm thấy ảnh test nào trong thư mục test_image/")
    print("💡 Hãy kiểm tra lại repository hoặc upload ảnh của bạn")

In [ ]:
# Test với một ảnh cụ thể
if test_images:
    # Chọn ảnh để test (thay đổi index nếu muốn test ảnh khác)
    test_index = 0  # Thay số này để chọn ảnh khác
    
    if test_index < len(test_images):
        selected_image = test_images[test_index]
        
        print(f"🚀 TESTING: {selected_image}")
        print("=" * 60)
        
        # Nhận dạng biển số
        result_img, detected_plates = detect_license_plate(selected_image, show_details=True)
        
        if result_img is not None:
            print("\n" + "=" * 60)
            print(f"🎯 KẾT QUẢ CUỐI CÙNG: {detected_plates if detected_plates else 'KHÔNG NHẬN DẠNG ĐƯỢC'}")
            print("=" * 60)
            
            # Hiển thị kết quả
            display_results(selected_image, result_img, detected_plates)
        else:
            print("❌ Không thể xử lý ảnh")
    else:
        print(f"❌ Index {test_index} không hợp lệ. Chọn từ 0 đến {len(test_images)-1}")
else:
    print("❌ Không có ảnh test để thử nghiệm")

## 📤 6. Upload và test ảnh của bạn

In [ ]:
from google.colab import files
import io

def upload_and_test():
    """
    Upload ảnh từ máy tính và test nhận dạng biển số
    """
    print("📤 Chọn file ảnh từ máy tính...")
    print("💡 Hỗ trợ: .jpg, .jpeg, .png")
    
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\n🔍 Đang xử lý: {filename}")
        print("=" * 50)
        
        # Lưu file tạm thời
        with open(filename, 'wb') as f:
            f.write(uploaded[filename])
        
        # Nhận dạng biển số
        result_img, detected_plates = detect_license_plate(filename, show_details=True, save_result=True)
        
        if result_img is not None:
            print("\n" + "=" * 50)
            print(f"🎯 KẾT QUẢ: {detected_plates if detected_plates else 'KHÔNG NHẬN DẠNG ĐƯỢC'}")
            print("=" * 50)
            
            # Hiển thị kết quả
            display_results(filename, result_img, detected_plates)
        else:
            print("❌ Không thể xử lý ảnh này")
        
        print("\n" + "-" * 50)

# Uncomment dòng dưới để chạy upload
# upload_and_test()

## 🔄 7. Test hàng loạt (Batch Testing)

In [ ]:
def batch_test_all_images():
    """
    Test tất cả ảnh trong thư mục test_image
    """
    if not test_images:
        print("❌ Không có ảnh test nào")
        return
    
    print(f"🚀 BẮT ĐẦU BATCH TEST - {len(test_images)} ảnh")
    print("=" * 70)
    
    results = []
    successful_count = 0
    total_plates_found = 0
    
    for i, img_path in enumerate(test_images, 1):
        print(f"\n📸 [{i}/{len(test_images)}] {os.path.basename(img_path)}")
        
        # Nhận dạng (không hiển thị chi tiết)
        result_img, detected_plates = detect_license_plate(img_path, show_details=False)
        
        if result_img is not None:
            plate_count = len(detected_plates) if detected_plates else 0
            results.append({
                'file': os.path.basename(img_path),
                'plates': detected_plates,
                'count': plate_count,
                'success': plate_count > 0
            })
            
            if plate_count > 0:
                successful_count += 1
                total_plates_found += plate_count
                print(f"   ✅ Phát hiện: {detected_plates}")
            else:
                print(f"   ❌ Không nhận dạng được")
        else:
            print(f"   ❌ Lỗi xử lý ảnh")
            results.append({
                'file': os.path.basename(img_path),
                'plates': None,
                'count': 0,
                'success': False
            })
    
    # Tổng kết
    print("\n" + "=" * 70)
    print("📊 TỔNG KẾT BATCH TEST:")
    print("=" * 70)
    print(f"🖼️  Tổng số ảnh test: {len(test_images)}")
    print(f"✅ Nhận dạng thành công: {successful_count}")
    print(f"❌ Không nhận dạng được: {len(test_images) - successful_count}")
    print(f"🚗 Tổng số biển số tìm thấy: {total_plates_found}")
    print(f"📈 Tỷ lệ thành công: {successful_count/len(test_images)*100:.1f}%")
    
    # Chi tiết kết quả
    print("\n📋 CHI TIẾT KẾT QUẢ:")
    for result in results:
        status = "✅" if result['success'] else "❌"
        plates_text = str(result['plates']) if result['plates'] else "Không nhận dạng được"
        print(f"   {status} {result['file']}: {plates_text}")
    
    return results

# Uncomment dòng dưới để chạy batch test
# batch_results = batch_test_all_images()

## 📝 8. Hướng dẫn sử dụng

### 🎯 Cách sử dụng notebook này:

1. **Chạy setup** (Cells 1-3): Clone repo, cài đặt dependencies, import libraries
2. **Load models** (Cell 4): Đợi models load xong (có thể mất 2-3 phút)
3. **Test ảnh mẫu** (Cell 6): Thay đổi `test_index` để test ảnh khác
4. **Upload ảnh** (Cell 7): Uncomment `upload_and_test()` để upload ảnh của bạn
5. **Batch test** (Cell 8): Uncomment `batch_test_all_images()` để test tất cả ảnh

### 💡 Tips:
- **Chọn GPU Runtime** để tăng tốc độ xử lý
- **Ảnh rõ nét** sẽ cho kết quả tốt hơn
- **Biển số không bị che khuất** sẽ dễ nhận dạng hơn
- **Góc chụp thẳng** cho kết quả tốt nhất

### 🔧 Troubleshooting:
- **Models không load được**: Restart Runtime và chạy lại
- **Lỗi import**: Kiểm tra cấu trúc thư mục
- **Không nhận dạng được**: Thử ảnh khác hoặc ảnh rõ nét hơn

---
**🚗 Chúc bạn sử dụng thành công! 🎉**